In [12]:
from tkinter import *
from enum import Enum
import tkinter.messagebox
import math
import time

In [13]:
# Graph class to store adjacency lists of 
# possible edges excluding obstacles

class Graph(object):
    def __init__(self):
        self.nodes = set()
        self.edges = {}
        self.distances = {}
    def add_node(self, value):
        self.nodes.add(value)
    
    # Adds new edge from node to other 
    # and from other to node - to make unweighted graph
    def add_edge(self, from_node, to_node):
        self._add_edge(from_node, to_node)
        self._add_edge(to_node, from_node)

    def _add_edge(self, from_node, to_node):
        self.edges.setdefault(from_node, [])
        self.edges[from_node].append(to_node)
        self.distances[(from_node, to_node)] = 1

    # Dijkstra algorithm set initial node as visited others nonvisited
    # Traverses edges while adding shortest path value to the current path value
    def dijkstra(self, initial_node):
        visited = {initial_node: 0}
        current_node = initial_node
        path = {}

        nodes = set(self.nodes)

        while nodes:
            min_node = None
            for node in nodes:
                if node in visited:
                    if min_node is None:
                        min_node = node
                    elif visited[node] < visited[min_node]:
                        min_node = node

            if min_node is None:
                break

            nodes.remove(min_node)
            cur_wt = visited[min_node]

            for edge in self.edges[min_node]:
                wt = cur_wt + self.distances[(min_node, edge)]
                if edge not in visited or wt < visited[edge]:
                    visited[edge] = wt
                    path[edge] = min_node

        return visited, path

    #Stores the route of the shortest path from Node x to Node y
    def route(self, x, y):
        distances, paths = self.dijkstra(x)
        route = [y]

        while y != x:
            route.append(paths[y])
            y = paths[y]

        route.reverse()
        return route

In [14]:
#Cell class to store a state's 
#possible movements, distance to another point
class Cell:  
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def right(self):
        return Cell(self.x,self.y+1)
    def left(self):
        return Cell(self.x,self.y-1)
    def up(self):
        return Cell(self.x-1,self.y)
    def down(self):
        return Cell(self.x+1,self.y)
    def distance(self,other):
        return math.sqrt(((other.x-self.x)**2)+((other.y-self.y)**2))
    def isEqual(self,other):
        return self.x==other.x and self.y==other.y
    
class Model:
    def __init__(self,grid_width,grid_height):
        #grid_width and grid_height specify the simulation environement
        self.grid_width = grid_width
        self.grid_height = grid_height
        self.empty = " "
        self.ped = "P"
        self.obs = "O"
        self.tar = "T"
        #obstacles in the simulation
        self.obstacles = [Cell(3,1),Cell(3,2),Cell(4,3),Cell(5,3),Cell(6,3),Cell(7,2),Cell(7,1)]
        #pedestrians in the simulation
        self.pedestrians = [Cell(5,0)]
        # Boolean array to check 
        # if a pedestrian reached the target
        self.reachedPedestrians = [False for i in range(len(self.pedestrians))]
        self.targets = [Cell(5,9)]
        self.grid = self.createGrid()
        self.placeStates()
        ##self.drawGrid()
        
        
    def cellToNumber(self,cell):
        return cell.y + cell.x*self.grid_width
    def numberToCell(self,number):
        return Cell(number/self.grid_width,number%self.grid_width)
    def dijkstra(self):
        g = Graph()
        g.nodes = set(range(self.grid_height*self.grid_width))
        #Adding edges to the graph
        for i in range(self.grid_height):
            for j in range(self.grid_width):
                if self.grid[i][j] != self.obs and self.isValidCell(Cell(i,j).right()):
                    num1 = self.cellToNumber(Cell(i,j))
                    num2 = self.cellToNumber(Cell(i,j).right())
                    g.add_edge(num1,num2)
                if self.grid[i][j] != self.obs and self.isValidCell(Cell(i,j).down()):
                    num1 = self.cellToNumber(Cell(i,j))
                    num2 = self.cellToNumber(Cell(i,j).down())
                    g.add_edge(num1,num2)
        targetNum = self.cellToNumber(self.targets[0])
        pedestrianNums = []
        routes = []
        for p in self.pedestrians:
            numRoutes = g.route(self.cellToNumber(p),targetNum)
            #if last element of the shortest path route is target, pedestrian reaches target
            if numRoutes[-1]==targetNum: 
                print("Pedestrian reached the target")
            else:
                print("Pedestrian didn't reacht the target")
            
        
        
    #Creates initial simulation with empty cells
    def createGrid(self):
        return [ [self.empty]*self.grid_width for i in range(self.grid_height)]
    def placeStates(self):
        for p in self.pedestrians:
            self.grid[p.x][p.y] = self.ped
        for p in self.obstacles:
            self.grid[p.x][p.y] = self.obs
        for p in self.targets:
            self.grid[p.x][p.y] = self.tar
    #Rudimentary cost function
    #Initially obstacle and other pedestrians are set to penalty of 50
    #0 penalty for empty and target cells
    def cost(self,cell):
        if self.grid[cell.x][cell.y] == self.obs:
            return 50
        elif self.grid[cell.x][cell.y] == self.ped:
            return 50
        else:
            return 0
    
    #Method checks if all pedestrians reach a target.
    def allreached(self):
        for p in self.reachedPedestrians:
            if not p:
                return False
        return True
    #Method checks if all pedestrians reach a target.
    def update(self,p,shortestCell,i,reached):
        self.grid[p.x][p.y] = self.empty
        self.grid[shortestCell.x][shortestCell.y] = self.ped
        self.pedestrians[i].x = shortestCell.x
        self.pedestrians[i].y = shortestCell.y
        if reached:
            self.reachedPedestrians[i] = True
    #Simulates each pedestrian one step 
    # according to shortest distance move
    def simulateOneStep(self):
        for i in range(len(self.pedestrians)):
            if not self.reachedPedestrians[i]:
                p = self.pedestrians[i]
                shortestCell,reached = self.findShortestMove(p)
                self.update(p,shortestCell,i,reached)
    #Checks if a cell is in the grid     
    def isValidCell(self,p):
        return p.x>=0 and p.x<self.grid_height and p.y>=0 and p.y<self.grid_width
    # Finds costs of possible moves with additional penalties
    # returns cell with smallest cost to the current pedestrian
    # returns reached: True if the current pedestrian reached the target
    def findShortestMove(self,p):
        currentCost = p.distance(self.targets[0])
        shortestCell = p
        for target in self.targets:
            if self.isValidCell(p.left()) and p.left().distance(target)+self.cost(p.left()) < currentCost:
                currentCost = p.left().distance(target)+self.cost(p.left())
                shortestCell = p.left()
            if self.isValidCell(p.right()) and p.right().distance(target)+self.cost(p.right()) < currentCost:
                currentCost = p.right().distance(target)+self.cost(p.right())
                shortestCell = p.right()
            if self.isValidCell(p.up()) and p.up().distance(target)+self.cost(p.up()) < currentCost:
                currentCost = p.up().distance(target)+self.cost(p.up())
                shortestCell = p.up()
            if self.isValidCell(p.down()) and p.down().distance(target)+self.cost(p.down()) < currentCost:
                currentCost = p.down().distance(target)+self.cost(p.down())
                shortestCell = p.down()
        # If pedestrian reaches target cost for getting onto target should be 1
        if currentCost == 1:
            reached = True
        else:
            reached = False
        return shortestCell,reached


In [15]:
#Method to display simulation grid
def display(simulation):
    boxSize=16
    for i in range(simulation.grid_height):
        for j in range(simulation.grid_width):
            if simulation.grid[i][j] == simulation.ped:
                canvas.create_rectangle(50+boxSize*j, 25+boxSize*i, 50+boxSize*(j+1), 25+boxSize*(i+1), fill="red",outline="black")
                canvas.create_text(50+boxSize*j+boxSize/2,25+boxSize*i+boxSize/2,text =simulation.ped)
            elif simulation.grid[i][j] == simulation.obs:
                canvas.create_rectangle(50+boxSize*j, 25+boxSize*i, 50+boxSize*(j+1), 25+boxSize*(i+1), fill="purple",outline="black")
                canvas.create_text(50+boxSize*j+boxSize/2,25+boxSize*i+boxSize/2,text =simulation.obs)
            elif simulation.grid[i][j] == simulation.tar:
                canvas.create_rectangle(50+boxSize*j, 25+boxSize*i, 50+boxSize*(j+1), 25+boxSize*(i+1), fill="goldenrod",outline="black")
                canvas.create_text(50+boxSize*j+boxSize/2,25+boxSize*i+boxSize/2,text =simulation.tar)
            else:
                canvas.create_rectangle(50+boxSize*j, 25+boxSize*i, 50+boxSize*(j+1), 25+boxSize*(i+1), fill="white",outline="black")
                canvas.create_text(50+boxSize*j+boxSize/2,25+boxSize*i+boxSize/2,text =simulation.empty)

In [16]:
grid_width = 10
grid_height = 10

simulation = Model(grid_width,grid_height)
root = tkinter.Tk()
root.title("Cellular Automaton")
root.resizable(width=False,height=False)
root.geometry("1000x1000")
canvas = tkinter.Canvas(root, width=550, height=550)
currentGrid = simulation.grid
display(simulation)

canvas.grid(row = 0, column = 10, sticky = E, pady = 2)
#Simulates the grid until either each pedestrian reaches the target
#or gets stuck in a constant position
#Displays the final states of simulation
def finalStep():
    while not simulation.allFinished():
        time.sleep(2)
        nextStep()
#Method to simulate one step on GUI
def nextStep():
    canvas.delete("all")
    simulation.simulateOneStep()
    currentGrid = simulation.grid
    display(simulation)
#Removes a pedestrian, obstacle or target from the position received by user input
def emptyCell():
    cellToDelete = Cell(rowNum.get(),colNum.get())
    for i in range(len(simulation.pedestrians)):
        if simulation.pedestrians[i].isEqual(cellToDelete):
            del simulation.pedestrians[i]
            del simulation.finishedPedestrians[i]
            break
    for i in range(len(simulation.obstacles)):
        if simulation.obstacles[i].isEqual(cellToDelete):
            del simulation.obstacles[i]
            break
    for i in range(len(simulation.targets)):
        if simulation.targets[i].isEqual(cellToDelete):
            del simulation.targets[i]
            break
    simulation.grid[cellToDelete.x][cellToDelete.y] = simulation.empty
    
    display(simulation)
#Adds new pedestrian to the position received by user input
def addPedestrian():
    cellToAdd = Cell(rowNum.get(),colNum.get())
    simulation.pedestrians.append(cellToAdd)
    simulation.grid[cellToAdd.x][cellToAdd.y] = simulation.ped
    simulation.finishedPedestrians.append(False)
    display(simulation)
#Adds new obstacle to the position received by user input
def addObstacle():
    cellToAdd = Cell(rowNum.get(),colNum.get())
    simulation.obstacles.append(cellToAdd)
    simulation.grid[cellToAdd.x][cellToAdd.y] = simulation.obs
    display(simulation)
#Adds new target to the position received by user input
def addTarget():
    cellToAdd = Cell(rowNum.get(),colNum.get())
    simulation.targets.append(cellToAdd)
    simulation.grid[cellToAdd.x][cellToAdd.y] = simulation.tar
    display(simulation)
    
#Buttons for GUI
clickButton = Button(root, text = "NEXT STEP", command = nextStep).grid(row = 0, column = 30)
clickButton2 = Button(root, text = "FINAL STEP", command = finalStep).grid(row = 1, column = 30)
rowNum = tkinter.IntVar()
colNum = tkinter.IntVar()
rowLabel = Label(root, text = 'Row Number [0-9]').grid(row = 3, column = 30)
rowEntry = Entry(root,textvariable = rowNum).grid(row = 4, column = 30)
colLabel = Label(root, text = 'Column Number [0-9]').grid(row = 5, column = 30)
colEntry=Entry(root, textvariable = colNum).grid(row = 6, column = 30)
addPedestrian = Button(root, text = "Add Pedestrian", command = addPedestrian).grid(row = 7, column = 30)
addObstacle = Button(root, text = "Add Obstacle", command = addObstacle).grid(row = 8, column = 30)
addTarget = Button(root, text = "Add Target", command = addTarget).grid(row = 9, column = 30)
emptyCell = Button(root, text = "Delete Cell", command = emptyCell).grid(row = 10, column = 30)
root.mainloop()

In [17]:

# Prints if pedestrians reache the target or not.
simulation.dijkstra()

Pedestrian reached the target
